In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the labeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = dsp.labeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['trend','groupby'], 'year':'trend', 'driver_gender':['trend','groupby'], 'driver_race':['trend','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'trend', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'trend', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical'}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops','contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v
/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_all_dpgmm()
labeled_df.df.head()

/home/smb/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,search_conducted_false_search_conducted_true_dpgmm,search_conducted_false_search_conducted_rate_dpgmm,search_conducted_false_contraband_found_false_dpgmm,search_conducted_false_num_stops_dpgmm,search_conducted_true_search_conducted_rate_dpgmm,search_conducted_true_contraband_found_false_dpgmm,search_conducted_true_num_stops_dpgmm,search_conducted_rate_contraband_found_false_dpgmm,search_conducted_rate_num_stops_dpgmm,contraband_found_false_num_stops_dpgmm
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,0,0,6,0,0,7,0,4,1,0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,0,6,6,0,6,7,0,7,6,0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,0,5,6,0,0,7,0,5,7,0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,0,5,6,0,0,7,0,5,7,0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,0,5,6,0,0,7,0,5,7,0


In [11]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [12]:
corrobj = dsp.all_pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

['year',
 'search_conducted_rate',
 'contraband_found_rate',
 'hit_rate',
 'num_stops']

In [13]:
rankobj = dsp.mean_rank_trend()
linreg_obj = dsp.linear_trend()

# Computing Trends on a labeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [14]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,subgroup_trend_quality,group_feat,subgroup,trend_type,agg_trend,agg_trend_quality
0,year,search_conducted_rate,-0.247018,-0.247018,state,AZ,pearson_corr,-0.03903,-0.03903
1,year,search_conducted_rate,-0.414566,-0.414566,state,CO,pearson_corr,-0.03903,-0.03903
2,year,search_conducted_rate,0.118238,0.118238,state,CT,pearson_corr,-0.03903,-0.03903
3,year,search_conducted_rate,-0.199765,-0.199765,state,IL,pearson_corr,-0.03903,-0.03903
4,year,search_conducted_rate,-0.603026,-0.603026,state,MA,pearson_corr,-0.03903,-0.03903


Now we can use a list of objects and apply multiple trends

In [15]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:117: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:119: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality
100,search_conducted_rate,driver_gender,search_conducted_rate_contraband_found_false_d...,1,"[F, M]",0.0636,rank_trend,"[F, M]",0.2519
343,contraband_found_rate,driver_gender,search_conducted_true_num_stops_dpgmm,11,[M],NaN,rank_trend,"[F, M]",0.1797
1350,num_stops,state,search_conducted_false_search_conducted_rate_d...,5,"[VT, TX, WI, RI, CT, AZ, WA, MA, IL, MD, SC, NC]",0.2276,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209
663,hit_rate,driver_race,search_conducted_false_contraband_found_false_...,8,[White],NaN,rank_trend,"[Asian, Hispanic, Other, Black, White]",0.0023
925,num_stops,driver_race,search_conducted_true_search_conducted_rate_dpgmm,2,"[Other, Black, Hispanic]",0.0645,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.4996
382,contraband_found_rate,driver_race,driver_gender,F,"[Asian, White, Hispanic, Black, Other]",0.2077,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.0716
1138,contraband_found_rate,state,search_conducted_false_num_stops_dpgmm,1,"[NC, TX]",0.8819,rank_trend,"[NC, CO, MD, WA, CT, SC, TX, WI, IL, MA, VT, A...",0.0201
1177,contraband_found_rate,state,search_conducted_true_num_stops_dpgmm,8,"[WA, SC, TX]",0.5500,rank_trend,"[NC, CO, MD, WA, CT, SC, TX, WI, IL, MA, VT, A...",0.0201
52,search_conducted_rate,driver_gender,search_conducted_false_contraband_found_false_...,8,[M],NaN,rank_trend,"[F, M]",0.2519
682,hit_rate,driver_race,search_conducted_true_search_conducted_rate_dpgmm,4,[White],NaN,rank_trend,"[Asian, Hispanic, Other, Black, White]",0.0023


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [16]:
pd.unique(labeled_df.result_df['trend_type'])

array(['rank_trend', 'lin_reg'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [17]:
labeled_df.trend_list

In [18]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance()

labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
526,hit_rate,driver_gender,search_conducted_false_search_conducted_true_d...,10,[M],NaN,rank_trend,"[F, M]",0.0433,NaN
707,hit_rate,driver_race,search_conducted_true_num_stops_dpgmm,9,"[Hispanic, White]",0.5774,rank_trend,"[Asian, Hispanic, Other, Black, White]",0.0023,0.47
71,search_conducted_rate,driver_gender,search_conducted_true_search_conducted_rate_dpgmm,4,[M],NaN,rank_trend,"[F, M]",0.2519,NaN
432,contraband_found_rate,driver_race,search_conducted_false_num_stops_dpgmm,11,"[Asian, Hispanic, White, Black, Other]",0.0240,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.0716,0.60
281,contraband_found_rate,driver_gender,search_conducted_false_search_conducted_true_d...,10,[M],NaN,rank_trend,"[F, M]",0.1797,NaN
332,contraband_found_rate,driver_gender,search_conducted_true_num_stops_dpgmm,0,"[F, M]",0.1094,rank_trend,"[F, M]",0.1797,0.00
546,hit_rate,driver_gender,search_conducted_false_num_stops_dpgmm,1,"[M, F]",0.6831,rank_trend,"[F, M]",0.0433,2.00
39,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_rate_d...,2,"[F, M]",0.2866,rank_trend,"[F, M]",0.2519,0.00
885,num_stops,driver_race,search_conducted_false_search_conducted_true_d...,3,[White],NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.4996,0.68
165,search_conducted_rate,driver_race,search_conducted_false_contraband_found_false_...,0,"[White, Asian, Black, Hispanic, Other]",0.0602,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.2772,0.60


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [19]:
labeled_df.trend_list[0].trend_precompute

{'rank_trend_agg_trend_contraband_found_rate_driver_gender': driver_gender
 F    0.003184
 M    0.006874
 dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_driver_race': driver_race
 Asian       0.002592
 White       0.004982
 Other       0.005700
 Hispanic    0.006455
 Black       0.008367
 dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_state': state
 NC    0.001062
 CO    0.002303
 MD    0.003503
 WA    0.004353
 CT    0.005716
 SC    0.006382
 TX    0.006541
 WI    0.007934
 IL    0.008215
 MA    0.008255
 VT    0.009567
 AZ    0.009980
 RI    0.013719
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_driver_gender': driver_gender
 F    0.243620
 M    0.256954
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_driver_race': driver_race
 Asian       0.165508
 Hispanic    0.193965
 Other       0.195967
 Black       0.269228
 White       0.279382
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_state': state
 WA    0.135670
 NC    0.146822
 IL    0.205232
 AZ    0

# Filtering

In [20]:
help(labeled_df.get_trend_row)

Help on method get_trend_row in module detect_simpsons_paradox.ranking_processing:

get_trend_row(feat1=None, feat2=None, group_feat=None, subgroup=None) method of detect_simpsons_paradox.labeled_dataframe.labeledDataFrame instance
    return a row of result_df based on the specified values
    
    Parameters
    -----------
    feat1 : str or  None
        trend variable name or None to include all
    feat2 : str or  None
        trend variable name or None to include all
    group_feat : str or  None
        groupoby variable name or None to include all
    subgroup : str or  None
        value of groupby_feat or or None to include all



So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [21]:
labeled_df.get_trend_row(subgroup='Black')

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
14,search_conducted_rate,driver_gender,driver_race,Black,"[F, M]",0.314600,rank_trend,"[F, M]",0.251900,0.000000
259,contraband_found_rate,driver_gender,driver_race,Black,"[F, M]",0.241300,rank_trend,"[F, M]",0.179700,0.000000
504,hit_rate,driver_gender,driver_race,Black,"[F, M]",0.145900,rank_trend,"[F, M]",0.043300,0.000000
749,num_stops,driver_gender,driver_race,Black,"[F, M]",0.214000,rank_trend,"[F, M]",0.116400,0.000000
983,search_conducted_rate,state,driver_race,Black,"[CO, NC, MD, MA, SC, CT, TX, WI, VT, WA, RI, A...",0.513600,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100,0.360000
1096,contraband_found_rate,state,driver_race,Black,"[NC, CO, MD, WA, SC, CT, MA, TX, IL, AZ, RI, W...",0.475500,rank_trend,"[NC, CO, MD, WA, CT, SC, TX, WI, IL, MA, VT, A...",0.020100,0.230000
1209,hit_rate,state,driver_race,Black,"[WA, NC, IL, MD, AZ, SC, CT, TX, RI, WI, CO, M...",0.534600,rank_trend,"[WA, NC, IL, AZ, MD, SC, TX, CT, RI, MA, WI, C...",0.221000,0.100000
1322,num_stops,state,driver_race,Black,"[VT, RI, WI, CO, CT, AZ, MA, WA, MD, IL, NC, T...",0.768800,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,0.310000
1700,search_conducted_rate,num_stops,driver_race,Black,-127215,0.135286,lin_reg,-257201,0.065594,0.000228


We can also filter based on SP

In [22]:
labeled_df.get_SP_rows(thresh=.2)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2
34,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_true_d...,8,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
54,search_conducted_rate,driver_gender,search_conducted_false_contraband_found_false_...,10,"[M, F]",0.857100,rank_trend,"[F, M]",0.251900,2.000000,True
56,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,1,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
62,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,7,"[M, F]",0.115900,rank_trend,"[F, M]",0.251900,2.000000,True
67,search_conducted_rate,driver_gender,search_conducted_true_search_conducted_rate_dpgmm,0,"[M, F]",0.153900,rank_trend,"[F, M]",0.251900,2.000000,True
72,search_conducted_rate,driver_gender,search_conducted_true_search_conducted_rate_dpgmm,5,"[M, F]",0.113600,rank_trend,"[F, M]",0.251900,2.000000,True
74,search_conducted_rate,driver_gender,search_conducted_true_search_conducted_rate_dpgmm,7,"[M, F]",NaN,rank_trend,"[F, M]",0.251900,2.000000,True
82,search_conducted_rate,driver_gender,search_conducted_true_contraband_found_false_d...,6,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
88,search_conducted_rate,driver_gender,search_conducted_true_num_stops_dpgmm,1,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
94,search_conducted_rate,driver_gender,search_conducted_true_num_stops_dpgmm,7,"[M, F]",0.164600,rank_trend,"[F, M]",0.251900,2.000000,True


# Ranking

In [27]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_quality', 'trend_type', 'agg_trend',
       'agg_trend_quality', 'distance', 'SP_thresh_0.2', 'mean_view_distance',
       'sum_view_SP_thresh_0.2'],
      dtype='object')


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance,sum_view_SP_thresh_0.2
0,search_conducted_rate,num_stops,search_conducted_true_num_stops_dpgmm,9,3.89019e+08,0.953559,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
1,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,10,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
2,search_conducted_rate,num_stops,search_conducted_false_contraband_found_false_...,8,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
3,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,10,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
4,search_conducted_rate,num_stops,search_conducted_true_contraband_found_false_d...,9,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
5,search_conducted_rate,num_stops,contraband_found_false_num_stops_dpgmm,8,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
6,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,3,8.41198e+07,0.823077,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
7,search_conducted_rate,num_stops,search_conducted_true_contraband_found_false_d...,3,8.41198e+07,0.823077,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
8,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,8,3.43021e+07,0.796159,lin_reg,-257201,0.065594,179.999776,True,72.328757,51.0
9,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,1,3.43021e+07,0.796159,lin_reg,-257201,0.065594,179.999776,True,72.328757,51.0


In [ ]:
labeled_df.add_view_score('SP_thresh_0.2',agg_type='sum',colored=False)

In [26]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh_0.2','SP_thresh_0.2').head()

Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_quality', 'trend_type', 'agg_trend',
       'agg_trend_quality', 'distance', 'SP_thresh_0.2', 'mean_view_distance',
       'sum_view_SP_thresh_0.2'],
      dtype='object')


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance,sum_view_SP_thresh_0.2
365,num_stops,state,search_conducted_false_contraband_found_false_...,0,"[AZ, WA, IL, CT, RI, SC, VT, TX, MD, MA, CO, N...",0.2266,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,1.03,True,0.48115,101.0
366,num_stops,state,search_conducted_false_num_stops_dpgmm,11,"[CT, AZ, RI, SC, IL, WA, TX, MD, VT, MA, CO, N...",0.1233,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,0.97,True,0.48115,101.0
367,num_stops,state,search_conducted_true_contraband_found_false_d...,10,"[AZ, CT, RI, IL, WA, SC, WI, TX, MD, VT, MA, N...",0.1093,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,0.87,True,0.48115,101.0
368,num_stops,state,search_conducted_false_search_conducted_true_d...,5,"[CT, RI, AZ, IL, WA, SC, VT, TX, MD, MA, WI, C...",0.0348,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,0.79,True,0.48115,101.0
369,num_stops,state,search_conducted_true_search_conducted_rate_dpgmm,3,"[RI, IL, AZ, SC, VT, CT, WI, TX, MA, MD, WA, N...",0.2050,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,0.77,True,0.48115,101.0


In [25]:
labeled_df.result_df.head()

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance,sum_view_SP_thresh_0.2
0,search_conducted_rate,num_stops,search_conducted_true_num_stops_dpgmm,9,3.89019e+08,0.953559,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
1,search_conducted_rate,num_stops,search_conducted_false_search_conducted_true_d...,10,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
2,search_conducted_rate,num_stops,search_conducted_false_contraband_found_false_...,8,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
3,search_conducted_rate,num_stops,search_conducted_false_num_stops_dpgmm,10,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
4,search_conducted_rate,num_stops,search_conducted_true_contraband_found_false_d...,9,3.36447e+08,1.000000,lin_reg,-257201,0.065594,179.999777,True,72.328757,51.0
